In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import cv2
from IPython.display import display
import matplotlib.pyplot as plt
import PIL.Image as Image

TRAIN_PATH = 'data/train.csv'
TEST_PATH = 'data/test.csv'

train_df = pd.read_csv(TRAIN_PATH)
val_df = pd.read_csv(TEST_PATH)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [8]:
train_data = []
for i in range(train_df.shape[0]):
    img = train_df.loc[i].values[1:].reshape(28, 28).astype(np.uint8)
    label = train_df.loc[i].values[0]
    train_data.append([img, label])

val_data = []
for i in range(val_df.shape[0]):
    img = val_df.loc[i].values[:].reshape(28, 28).astype(np.uint8)
    val_data.append([img, label])
# img = train_df.loc[14].values[1:].reshape(28, 28).astype(np.uint8)

print(f'Train data size: {len(train_data)}')
print(f'Validation data size: {len(val_data)}')


Train data size: 42000
Validation data size: 28000


In [3]:
class simpleNN(nn.Module):
    def __init__(self):
        super(simpleNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # x = x.view(-1, 28*28)
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.softmax(x)
        return x

In [10]:
from torch.optim import Adam
model = simpleNN()
model = model.to(device)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()   
EPOCHS = 150
BATCH_SIZE = 1000
for epoch in range(EPOCHS):
    np.random.shuffle(train_data)
    for i in range(0, len(train_data), BATCH_SIZE):
        batch = train_data[i:i+BATCH_SIZE]
        imgs = np.array([item[0] for item in batch])
        labels = np.array([item[1] for item in batch])
        imgs = torch.tensor(imgs).float().to(device) / 255.0
        labels = torch.tensor(labels).long().to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{EPOCHS}, Training Loss: {loss.item()}')

Epoch 1/150, Loss: 1.9459493160247803
Epoch 2/150, Loss: 1.747284173965454
Epoch 3/150, Loss: 1.6974680423736572
Epoch 4/150, Loss: 1.681800365447998
Epoch 5/150, Loss: 1.6850788593292236
Epoch 6/150, Loss: 1.677911400794983
Epoch 7/150, Loss: 1.6619136333465576
Epoch 8/150, Loss: 1.6749321222305298
Epoch 9/150, Loss: 1.66920006275177
Epoch 10/150, Loss: 1.6550832986831665
Epoch 11/150, Loss: 1.5945029258728027
Epoch 12/150, Loss: 1.5878933668136597
Epoch 13/150, Loss: 1.5867183208465576
Epoch 14/150, Loss: 1.5853362083435059
Epoch 15/150, Loss: 1.5777753591537476
Epoch 16/150, Loss: 1.5764721632003784
Epoch 17/150, Loss: 1.5883588790893555
Epoch 18/150, Loss: 1.5611578226089478
Epoch 19/150, Loss: 1.5711122751235962
Epoch 20/150, Loss: 1.5824368000030518
Epoch 21/150, Loss: 1.5912481546401978
Epoch 22/150, Loss: 1.5719311237335205
Epoch 23/150, Loss: 1.5772271156311035
Epoch 24/150, Loss: 1.5618693828582764
Epoch 25/150, Loss: 1.5847477912902832
Epoch 26/150, Loss: 1.5748542547225952


In [ ]:
model.eval()
image_index = 10
num_correct = 0
with torch.no_grad():
    for i in range(len(val_data)):
        output = model.forward(torch.tensor(val_data[i][0]).float().unsqueeze(0).to(device) / 255.0)
        if (output.argmax() == val_data[i][1]):
            num_correct  1
        #     print(f"Predicted {output.argmax()}")
        # print(f"Predicted {output.argmax()}")
        # print(f"Actual {train_data[56][1]}")
        # plt.imshow(val_data[i][0], cmap='gray')
print(f'Validation accuracy: {num_correct/len(val_data)*100:.2f}%')

Validation accuracy: 9.98%
